In [1]:
from collections import Counter, OrderedDict
from pathlib import Path
import sys
import textwrap
from tqdm import tqdm
import torch
from typing import Dict, List, Set, Union

from transformers import Seq2SeqTrainingArguments

import unicodedata
import uuid

from aic_nlp_utils.json import read_jsonl, read_json, write_json, write_jsonl
from aic_nlp_utils.fever import fever_detokenize, import_fever_corpus_from_sqlite
from simpletransformers.seq2seq import Seq2SeqModel, Seq2SeqArgs
import stanza
# stanza.download("en")

sys.path.append('Claim_Generation')
from T5_QG import pipeline
from distractor_generation import Distractor_Generation

sys.path.append('Models')
from arguments import ModelArguments, DataTrainingArguments
from load import load_tokenizer_and_model, find_last_checkpoint

%load_ext autoreload
%autoreload 2

This notebook prepares data to train QG models for different languages.
The paper used T5 model from here: https://github.com/patil-suraj/question_generation

In [ ]:
def convert_and_fix_squad(fsrc, fdst):
    # converts SQUAD format to "linear" JSONL usable for training seq2seq model 
    data = read_json(fsrc)["data"]
    records = []
    for rec in tqdm(data):
        title = unicodedata.normalize("NFC", rec["title"])
        for par in rec["paragraphs"]:
            context = unicodedata.normalize("NFC", par["context"])
            for qas in par["qas"]:
                answer_set = set()
                for ans in qas["answers"]:
                    ans = ans["text"]
                    if ans[-1] in [".", ","]:
                        ans = ans[:-1]
                    answer_set.add(ans)
                answers = sorted(list(answer_set))
                for answer in answers:
                    question = qas["question"].strip()
                    question = question[0].upper() + question[1:]
                    question = question.replace("  ", " ")
                    if not (question.endswith("?") or question.endswith('?"')):
                        if not question.lower().startswith("name"):
                            if question[-1] in [".", ":", ">", "/"]: # Probably wrong parsing of original data
                                question = question[:-1] + "?"
                            else:
                                question += "?"
                    question = unicodedata.normalize("NFC", question)
                    answer = unicodedata.normalize("NFC", answer)
                    print(question)
                    records.append({"title": title, "context": context, "question": question, "answer": answer})
    write_jsonl(fdst, records, mkdir=True)

SQUAD_ROOT = Path("/mnt/data/factcheck/squad/squad-cs")
QG_ROOT = Path("/mnt/data/factcheck/qg/squad-cs")

convert_and_fix_squad(Path(SQUAD_ROOT, "dev-v1.1.json"), Path(QG_ROOT, "dev-v1.1.jsonl"))
convert_and_fix_squad(Path(SQUAD_ROOT, "train-v1.1.json"), Path(QG_ROOT, "train-v1.1.jsonl"))

In [2]:
model_args = ModelArguments(model_name_or_path="/home/drchajan/devel/python/FC/Zero-shot-Fact-Verification/experiments/qg/facebook/mbart-large-cc25_cs_CZ/BEST/checkpoint-32000")

In [3]:
tokenizer, model, data_collator = load_tokenizer_and_model(model_args, lang="cs_CZ", fp16=True)


In [4]:
data = read_jsonl("/mnt/data/factcheck/qg/squad-cs/dev-v1.1.json")

In [5]:
data[0]

{'title': 'Super_Bowl_50',
 'context': 'Super Bowl 50 byl zápas amerického fotbalu, který určil šampiona Národní fotbalové ligy (NFL) pro sezónu 2015. Šampion Americké fotbalové konference (AFC) Denver Broncos porazil šampiona Národní fotbalové konference (NFC) Carolinu Panthers 24:10 a získal třetí titul v Super Bowlu. Zápas se hrál 7. února 2016 na Leviho stadionu v San Francisco Bay Area v Santa Claře v Kalifornii. Protože se jednalo o 50. Super Bowl, liga zdůrazňovala "zlaté výročí" různými iniciativami se zlatou tematikou a také dočasně pozastavila tradici pojmenovávání každé hry Super Bowl římskými číslicemi (pod kterými by hra byla známá jako "Super Bowl L"), aby logo mohlo výrazně zobrazovat arabské číslice 50.',
 'question': 'Který tým NFL reprezentoval AFC na Super Bowlu 50?',
 'answer': 'Denver Broncos'}

In [34]:
def predict(model, tokenizer, inputs, max_source_length=1024, padding=False):
    model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True, return_tensors="pt")
    with torch.no_grad():
        Y = model.generate(**model_inputs, max_new_tokens=768)
        predictions = tokenizer.batch_decode(
            Y, skip_special_tokens=True, clean_up_tokenization_spaces=True
        )
    return predictions


sample = data[120]
answer = sample["answer"]
context = "Vzkaz na vojenské technice zaujal po sobotní části Pavlovy návštěvy Ukrajiny některá ukrajinská média. Píše o něm také kupříkladu agentura Unian, která zároveň informuje o Pavlově setkání s ukrajinskými vojáky v Dněpropetrovské oblasti, agentura Ukrinform, jež připomněla Pavlovo působení na vrcholné pozici v NATO, nebo server Obozrevatel. Ukrinform mimo jiné zaznamenal jednání prezidenta s místními činiteli o obnově Dněpropetrovské oblasti, nad níž Česko převzalo záštitu. Weby Hromadske nebo Jevropejska pravda upozornily na prezidentovo setkání s vysídlenými Ukrajinci. Ukrajinská média informovala už dříve o pátečním programu Pavla a Čaputové, kromě jejich oficiálních setkání s ukrajinskými činiteli si povšimla mimo jiné faktu, že státníci museli kvůli vzdušnému poplachu v pátek na čas do hotelového krytu. Ukrajinska pravda v souvislosti s návštěvou prezidentů Česka a Slovenska poznamenala, že vysoce postavení zahraniční představitelé od začátku ruské invaze jen zřídka zůstali na Ukrajině přes noc."
answer = "NATO"
print(textwrap.fill(context))
print(answer)
predict(model, tokenizer, [answer + "</s>" + context])

Vzkaz na vojenské technice zaujal po sobotní části Pavlovy návštěvy
Ukrajiny některá ukrajinská média. Píše o něm také kupříkladu agentura
Unian, která zároveň informuje o Pavlově setkání s ukrajinskými vojáky
v Dněpropetrovské oblasti, agentura Ukrinform, jež připomněla Pavlovo
působení na vrcholné pozici v NATO, nebo server Obozrevatel. Ukrinform
mimo jiné zaznamenal jednání prezidenta s místními činiteli o obnově
Dněpropetrovské oblasti, nad níž Česko převzalo záštitu. Weby
Hromadske nebo Jevropejska pravda upozornily na prezidentovo setkání s
vysídlenými Ukrajinci. Ukrajinská média informovala už dříve o
pátečním programu Pavla a Čaputové, kromě jejich oficiálních setkání s
ukrajinskými činiteli si povšimla mimo jiné faktu, že státníci museli
kvůli vzdušnému poplachu v pátek na čas do hotelového krytu.
Ukrajinska pravda v souvislosti s návštěvou prezidentů Česka a
Slovenska poznamenala, že vysoce postavení zahraniční představitelé od
začátku ruské invaze jen zřídka zůstali na Ukraj

['Která agentura připomněla Pavlovo působení na vrcholné pozici?']